In [ ]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg16 #torchvision.model 안에는 많은 모델이 저장 되어있다.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = vgg16(pretrained= True)


In [ ]:
print(device)

In [ ]:
model

In [ ]:
# 출력층 지정 dropout = 과적합 방지, 마지막 모델 계층은 10으로 지정, cifar 10이기 때문에
fc = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096, 2048),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(256, 10)
)

# 데이터 normalize 하는 이유 : 색깔 값이 데이터 판별에 영향을 미치기 때문에 색값을 지워주려고 노멀라이즈 한다.
# 데이터 평균 수치값은 어떻게 뽑나? 그거 한번 해보자
# 표준 편차 구하는방법

model.classifier = fc
model.to(device)
model

In [ ]:
#tqdm  진행률 프로세스 바

In [ ]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize # 편리성 덕분에 transform 많이 사용
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

In [ ]:
test_data = CIFAR10(root='./', train = True, download=True, transform=ToTensor())


In [ ]:
len(test_data)

In [ ]:
imgs = [item[0] for item in test_data]
len(imgs)

In [ ]:
imgs = torch.stack(imgs, dim = 0).numpy() # 평균값을 구하기 위해 numpy 형태로 변경, torch.stack 명령은 tensor들을 합치는 명령어

In [ ]:
imgs.shape

In [ ]:
mean_r = imgs[:, 0,:,:].mean()
mean_g = imgs[:, 1,:,:].mean()
mean_b = imgs[:, 2,:,:].mean()

print(mean_r, mean_g, mean_b)

In [ ]:
std_r = imgs[:, 0,:,:].std()
std_g = imgs[:, 1,:,:].std()
std_b = imgs[:, 2,:,:].std()

print(std_r, std_g, std_b)

In [ ]:
# 이미지 입력시 순차적으로 해야할 일들을 지정
transforms = Compose([
                      Resize(224),
                      RandomCrop((224,224), padding=4),
                      RandomHorizontalFlip(),
                      ToTensor(),
                      Normalize(mean=(0.4914,0.4822, 0.4465), std=(0.247, 0.243, 0.261))
                      ])

In [ ]:
train_data = CIFAR10(
    root='./',
    train = True,
    download=True,
    transform = transforms
)

test_data = CIFAR10(
    root='./',
    train = False,
    download=True,
    transform = transforms
)


# DataLoader 편리한 패키지
train_loader = DataLoader(train_data, batch_size = 32, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 32, shuffle=True)

In [ ]:
lr = 0.004
optim = Adam(model.parameters(), lr = lr)


for epoch in range(30):
    iterator = tqdm.tqdm(train_loader)

    for data,label in iterator:
        optim.zero_grad()

        # to(device)에 올라가 있기 때문에 label 값고 같이 올려줘야 한다.
        preds = model(data.to(device))
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        optim.step()

        iterator.set_description(f'epoch:{epoch+1} loss:{loss.item()}')

#모델의 weight값만 저장
torch.save(model.state_dict(), 'cifar10_pre.pth')

In [ ]:
#map_location 장치 설정
dict_model = torch.load('cifar10_pre.pth', map_location = device)
model.load_state_dict(dict_model)

num_corr = 0

with torch.no_grad():
    # nograde - 더이상 오토그레이드를 하지 않겠다
    for data, label in test_loader:
        output = model(data.to(device))
        preds = output.data.max(1)[1]
        corr = preds.eq(label.to(device).data).sum().item()
        num_corr += corr

    print(num_corr/len(test_loader))
